1. Load Data (stratified sample)
2. train-val-test split
4. clean/tokenize, not stop, not lemma
5. convert to tf.dataset
6. run models

In [ ]:
# loading sample Data
%load_ext autoreload
%autoreload 2


In [14]:
import numpy as np
import pandas as pd
import tensorflow as tf
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, Sequential
import time
from tqdm.notebook import tqdm
import nltk
nltk.download('averaged_perceptron_tagger', quiet=True)
nltk.download('wordnet', quiet=True)

def prepare_tf_dataset(X, y, batch_size, is_training=False):
    """
    Prepares a TensorFlow dataset for efficient training or evaluation.
    """
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    if is_training:
        dataset = dataset.shuffle(10000)  # Shuffle only if dataset is for training
    return dataset.batch(batch_size).cache().prefetch(tf.data.AUTOTUNE)

def load_data(path, include_validation=False, sample_size=0):
    """Load review data from a CSV file, with optional sampling and validation split."""
    df = pd.read_csv(path)[['Review_Title', 'Review', 'Recommended']]
    if sample_size > 0:
        df = df.sample(sample_size)
    
    X = df['Review_Title'] + ' ' + df['Review']
    y = df['Recommended'].map({'yes': 1, 'no': 0})
    
    if include_validation:
        return split_data_with_validation(X, y)
    else:
        return train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

def split_data_with_validation(X, y):
    """Split data into training, validation, and test sets."""
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)
    return X_train, X_val, X_test, y_train, y_val, y_test

def get_wordnet_pos(treebank_tag):
    """Map POS tag to first character lemmatize() accepts."""
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def clean_text(review, stop_words=None, lemmatize=True):
    """Clean and preprocess a single review text."""
    tokenizer = RegexpTokenizer(r"([a-zA-Z]+(?:’[a-z]+)?)")
    lemmatizer = WordNetLemmatizer() if lemmatize else None
    
    tokens = tokenizer.tokenize(review.lower())
    if lemmatize:
        pos_tags = pos_tag(tokens)
        tokens = [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in pos_tags]
    if stop_words:
        tokens = [word for word in tokens if word not in stop_words]
    
    return ' '.join(tokens)

def preprocess_texts(reviews, stop_words=None, lemmatize=False):
    """Apply text cleaning and preprocessing to a list of texts."""
    return [clean_text(review, stop_words=stop_words, lemmatize=lemmatize) for review in reviews]

def load_and_prepare_data(path, include_validation=True, sample_size=5000, 
                          stop_words=None, lemmatize=False, 
                          max_tokens=10000, percentile_len=0.9, batch_size=64):
    """
    Load, clean, and prepare data for training and validation.
    """
    # Load the data with validation split and optional sampling
    X_train, X_val, X_test, y_train, y_val, y_test = load_data(
        path, include_validation=include_validation, sample_size=sample_size)
    
    # Clean the text data
    X_train = preprocess_texts(X_train, stop_words=stop_words, lemmatize=lemmatize)
    X_val = preprocess_texts(X_val, stop_words=stop_words, lemmatize=lemmatize)
    
    # Calculating percentile sequence length and vocabulary size for training set
    lengths = pd.Series([len(review.split()) for review in X_train])
    sequence_length = int(lengths.quantile(percentile_len))
    vocab_size = min(max_tokens, len(set(word for review in X_train for word in review.split())))
    
    # Initialize and adapt the TextVectorization layer
    text_vectorization = tf.keras.layers.TextVectorization(
        standardize=None,
        max_tokens=vocab_size,
        output_mode='int',
        output_sequence_length=sequence_length)
    
    text_vectorization.adapt(X_train)
    X_train = text_vectorization(X_train)
    X_val = text_vectorization(X_val)
    
    # Prepare the datasets
    train_ds = prepare_tf_dataset(X_train, y_train, batch_size, is_training=True)
    val_ds = prepare_tf_dataset(X_val, y_val, batch_size)

    return train_ds, val_ds, vocab_size, sequence_length, batch_size

def add_rnn_layer(model, units, rnn_type='gru', bidirectional=False, return_sequences=False):
    LayerClass = layers.GRU if rnn_type == 'gru' else layers.LSTM
    layer = LayerClass(units, return_sequences=return_sequences)
    if bidirectional:
        layer = layers.Bidirectional(layer)
    model.add(layer)

def build_rnn_model(rnn_layers, dense_layers, recurrent_type, bi_directional, dropout_rate, units, sequence_length, vocab_size):

    model = Sequential()
    model.add(layers.Embedding(input_dim=vocab_size, output_dim=units, input_length=sequence_length))

    for i in range(rnn_layers):
        add_rnn_layer(model, units, rnn_type=recurrent_type, bidirectional=bi_directional, 
                      return_sequences=(i < rnn_layers - 1))
        model.add(layers.Dropout(dropout_rate))
        units = max(2, units // 2)

    for _ in range(dense_layers):
        model.add(layers.Dense(units, activation="relu"))
        model.add(layers.Dropout(dropout_rate))
        units = max(2, units // 2)

    model.add(layers.Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy', tf.keras.metrics.AUC(name="auc")])
    return model

def extract_performance_metrics(history, callbacks, duration):
    early_stopping = next(
        (cb for cb in callbacks if isinstance(cb, tf.keras.callbacks.EarlyStopping)), 
        None
    )
    if early_stopping and early_stopping.stopped_epoch > 0:
        adjusted_epoch = early_stopping.stopped_epoch - early_stopping.patience
        max_epoch_index = len(history.history['loss']) - 1
        best_epoch = max(0, min(adjusted_epoch, max_epoch_index))
    else:
        best_epoch = len(history.history['loss']) - 1

    metrics = {
        'loss': history.history['loss'][best_epoch],
        'val_loss': history.history['val_loss'][best_epoch],
        'val_accuracy': history.history.get('val_accuracy', [None])[best_epoch],
        'val_auc': history.history.get('val_auc', [None])[best_epoch],
        'duration': duration
    }
    return metrics

def train_and_evaluate(model_function, train_ds, val_ds, epochs):
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            min_delta=0.001,
            patience=5, 
            restore_best_weights=True,
            verbose=0
        )
    ]
    
    model = model_function()    
    start_time = time.time()
    history = model.fit(
        train_ds, 
        epochs=epochs, 
        validation_data=val_ds,
        callbacks=callbacks,
        verbose=0
    )
    duration = time.time() - start_time
    return extract_performance_metrics(history, callbacks, duration)

def calculate_average_metrics(runs, model_function, train_ds, val_ds, epochs):
    metrics_aggregate = {'loss': 0, 'val_loss': 0, 'val_accuracy': 0, 'val_auc': 0, 'duration': 0}

    for _ in range(runs):
        metrics = train_and_evaluate(model_function, train_ds, val_ds, epochs)
        for key in metrics_aggregate:
            metrics_aggregate[key] += metrics[key]

    return {key: val / runs for key, val in metrics_aggregate.items()}

In [2]:
# Load and Prepare the Data
train_ds, val_ds, vocab_size, sequence_length, batch_size = load_and_prepare_data(
    path='data/Airline_review.csv', 
    include_validation=True, 
    sample_size=5000, 
    stop_words=None, 
    lemmatize=False, 
    max_tokens=10000, 
    percentile_len=0.9, 
    batch_size=64
)

In [2]:
from itertools import product
import pandas as pd

def generate_configurations(param_dict):
    # Extract parameter names and their possible values
    param_names = list(param_dict.keys())
    param_values = list(param_dict.values())

    # Compute the Cartesian product of parameter values
    all_combinations = product(*param_values)

    # Construct a list of dictionaries for each combination
    configurations = [dict(zip(param_names, combination)) for combination in all_combinations]
   
    return configurations

In [8]:
import pandas as pd

train_ds, val_ds, vocab_size, sequence_length, batch_size = load_and_prepare_data(
    path='data/Airline_review.csv', 
    include_validation=True, 
    sample_size=1000, 
    stop_words=None, 
    lemmatize=False, 
    max_tokens=10000, 
    percentile_len=0.9, 
    batch_size=64
)
def run_experiment(recurrent_type, bi_directional, rnn_layers, dense_layers, runs, train_ds, val_ds, epochs, sequence_length, vocab_size):
    """
    Runs the experiment for a specific configuration and returns the average metrics.
    """
    model_function = lambda: build_rnn_model(
        rnn_layers=rnn_layers,
        dense_layers=dense_layers,
        recurrent_type=recurrent_type,
        bi_directional=bi_directional,
        dropout_rate=0.2,
        units=64,
        sequence_length=sequence_length,
        vocab_size=vocab_size
    )
    return calculate_average_metrics(runs, model_function, train_ds, val_ds, epochs)

# Configuration options
param_dict = {
    'recurrent_type': ['gru', 'lstm'],
    'bi_directional': [True, False],
    'rnn_layers': [1,2],
    'dense_layers': [1,2]
}

configurations = generate_configurations(param_dict)

results = []
runs = 3
epochs = 50

# Run experiments
for config in configurations:
    metrics = run_experiment(
        recurrent_type=config['recurrent_type'],
        bi_directional=config['bi_directional'],
        rnn_layers=config['rnn_layers'],
        dense_layers=config['dense_layers'],
        runs=runs,
        train_ds=train_ds,
        val_ds=val_ds,
        epochs=epochs,
        sequence_length=sequence_length,
        vocab_size=vocab_size
    )
    results.append({
        **metrics,
        **config  # Unpack configuration into the results
    })

# Create DataFrame and format
df = pd.DataFrame(results)
df = df.round({'loss': 4, 'val_loss': 4, 'val_accuracy': 4, 'val_auc': 4})
df['duration'] = df['duration'].round(0).astype(int)

df



Epoch 1/50

13/13 [==============================] - 4s 153ms/step - loss: 0.6753 - accuracy: 0.6388 - auc: 0.4666 - val_loss: 0.6498 - val_accuracy: 0.6500 - val_auc: 0.6470
Epoch 2/50
13/13 [==============================] - 1s 88ms/step - loss: 0.6397 - accuracy: 0.6488 - auc: 0.5771 - val_loss: 0.6289 - val_accuracy: 0.6500 - val_auc: 0.7758
Epoch 3/50
13/13 [==============================] - 1s 90ms/step - loss: 0.5823 - accuracy: 0.6488 - auc: 0.8588 - val_loss: 0.5289 - val_accuracy: 0.6800 - val_auc: 0.8960
Epoch 4/50
13/13 [==============================] - 1s 87ms/step - loss: 0.3722 - accuracy: 0.8425 - auc: 0.9442 - val_loss: 0.3812 - val_accuracy: 0.8100 - val_auc: 0.9222
Epoch 5/50
13/13 [==============================] - 1s 89ms/step - loss: 0.2215 - accuracy: 0.9350 - auc: 0.9699 - val_loss: 0.4468 - val_accuracy: 0.7400 - val_auc: 0.9244
Epoch 6/50
13/13 [==============================] - 1s 97ms/step - loss: 0.1683 - accuracy: 0.9563 - auc: 0.9898 - val_loss: 0.4552 

,loss,val_loss,val_accuracy,val_auc,duration,recurrent_type,bi_directional,rnn_layers,dense_layers
0,0.3393,0.3916,0.8367,0.9313,14,gru,True,1,1
1,0.3538,0.3969,0.8567,0.9319,16,gru,True,1,2
2,0.2811,0.3721,0.8233,0.9245,26,gru,True,2,1
3,0.4258,0.4184,0.7500,0.9302,27,gru,True,2,2
4,0.6138,0.6153,0.6500,0.5566,15,gru,False,1,1
5,0.6404,0.6358,0.6500,0.5241,12,gru,False,1,2
6,0.6354,0.6329,0.6500,0.5651,20,gru,False,2,1
7,0.6306,0.6320,0.6467,0.5455,23,gru,False,2,2
8,0.2308,0.3543,0.8567,0.9508,18,lstm,True,1,1
9,0.2731,0.3321,0.8733,0.9401,20,lstm,True,1,2


In [11]:
df.sort_values('val_loss')

,loss,val_loss,val_accuracy,val_auc,duration,recurrent_type,bi_directional,rnn_layers,dense_layers
9,0.2731,0.3321,0.8733,0.9401,20,lstm,True,1,2
10,0.3452,0.3396,0.8600,0.9400,26,lstm,True,2,1
8,0.2308,0.3543,0.8567,0.9508,18,lstm,True,1,1
2,0.2811,0.3721,0.8233,0.9245,26,gru,True,2,1
0,0.3393,0.3916,0.8367,0.9313,14,gru,True,1,1
1,0.3538,0.3969,0.8567,0.9319,16,gru,True,1,2
11,0.3957,0.4153,0.8033,0.9345,29,lstm,True,2,2
3,0.4258,0.4184,0.7500,0.9302,27,gru,True,2,2
14,0.5568,0.5495,0.7167,0.6789,32,lstm,False,2,1
12,0.6062,0.5935,0.6800,0.5921,17,lstm,False,1,1


In [13]:
df.sort_values('val_loss')

,loss,val_loss,val_accuracy,val_auc,duration,recurrent_type,bi_directional,rnn_layers,dense_layers
4,0.2238,0.2753,0.8920,0.9526,65,lstm,True,1,1
6,0.3036,0.2966,0.8660,0.9426,100,lstm,True,2,1
5,0.3040,0.3118,0.8653,0.9359,63,lstm,True,1,2
1,0.3542,0.3133,0.8667,0.9450,50,gru,True,1,2
3,0.4056,0.3188,0.8673,0.9363,88,gru,True,2,2
0,0.2950,0.3189,0.8593,0.9357,49,gru,True,1,1
7,0.3686,0.3464,0.8673,0.9344,105,lstm,True,2,2
2,0.3838,0.3534,0.8493,0.9289,85,gru,True,2,1


In [12]:
# Bidirectional it is! 1 parameter down!

train_ds, val_ds, vocab_size, sequence_length, batch_size = load_and_prepare_data(
    path='data/Airline_review.csv', 
    include_validation=True, 
    sample_size=5000, 
    stop_words=None, 
    lemmatize=False, 
    max_tokens=10000, 
    percentile_len=0.9, 
    batch_size=64
)
def run_experiment(recurrent_type, bi_directional, rnn_layers, dense_layers, runs, train_ds, val_ds, epochs, sequence_length, vocab_size):
    """
    Runs the experiment for a specific configuration and returns the average metrics.
    """
    model_function = lambda: build_rnn_model(
        rnn_layers=rnn_layers,
        dense_layers=dense_layers,
        recurrent_type=recurrent_type,
        bi_directional=bi_directional,
        dropout_rate=0.2,
        units=64,
        sequence_length=sequence_length,
        vocab_size=vocab_size
    )
    return calculate_average_metrics(runs, model_function, train_ds, val_ds, epochs)

# Configuration options
param_dict = {
    'recurrent_type': ['gru', 'lstm'],
    'bi_directional': [True],
    'rnn_layers': [1,2],
    'dense_layers': [1,2]
}

configurations = generate_configurations(param_dict)

results = []
runs = 3
epochs = 50

# Run experiments
for config in configurations:
    metrics = run_experiment(
        recurrent_type=config['recurrent_type'],
        bi_directional=config['bi_directional'],
        rnn_layers=config['rnn_layers'],
        dense_layers=config['dense_layers'],
        runs=runs,
        train_ds=train_ds,
        val_ds=val_ds,
        epochs=epochs,
        sequence_length=sequence_length,
        vocab_size=vocab_size
    )
    results.append({
        **metrics,
        **config  # Unpack configuration into the results
    })

# Create DataFrame and format
df = pd.DataFrame(results)
df = df.round({'loss': 4, 'val_loss': 4, 'val_accuracy': 4, 'val_auc': 4})
df['duration'] = df['duration'].round(0).astype(int)

df

Epoch 1/50
63/63 [==============================] - 9s 106ms/step - loss: 0.5989 - accuracy: 0.6765 - auc: 0.6463 - val_loss: 0.3529 - val_accuracy: 0.8580 - val_auc: 0.9264
Epoch 2/50
63/63 [==============================] - 6s 102ms/step - loss: 0.2929 - accuracy: 0.8810 - auc: 0.9404 - val_loss: 0.3281 - val_accuracy: 0.8580 - val_auc: 0.9313
Epoch 3/50
63/63 [==============================] - 7s 105ms/step - loss: 0.1663 - accuracy: 0.9380 - auc: 0.9799 - val_loss: 0.3539 - val_accuracy: 0.8760 - val_auc: 0.9405
Epoch 4/50
63/63 [==============================] - 7s 106ms/step - loss: 0.1046 - accuracy: 0.9640 - auc: 0.9917 - val_loss: 0.4708 - val_accuracy: 0.8860 - val_auc: 0.9360
Epoch 5/50
63/63 [==============================] - 7s 105ms/step - loss: 0.0796 - accuracy: 0.9740 - auc: 0.9947 - val_loss: 0.4634 - val_accuracy: 0.8700 - val_auc: 0.9258
Epoch 6/50
63/63 [==============================] - 7s 104ms/step - loss: 0.0705 - accuracy: 0.9772 - auc: 0.9962 - val_loss: 0.74

,loss,val_loss,val_accuracy,val_auc,duration,recurrent_type,bi_directional,rnn_layers,dense_layers
0,0.2950,0.3189,0.8593,0.9357,49,gru,True,1,1
1,0.3542,0.3133,0.8667,0.9450,50,gru,True,1,2
2,0.3838,0.3534,0.8493,0.9289,85,gru,True,2,1
3,0.4056,0.3188,0.8673,0.9363,88,gru,True,2,2
4,0.2238,0.2753,0.8920,0.9526,65,lstm,True,1,1
5,0.3040,0.3118,0.8653,0.9359,63,lstm,True,1,2
6,0.3036,0.2966,0.8660,0.9426,100,lstm,True,2,1
7,0.3686,0.3464,0.8673,0.9344,105,lstm,True,2,2


In [3]:
# Bidirectional it is! 1 parameter down!

train_ds, val_ds, vocab_size, sequence_length, batch_size = load_and_prepare_data(
    path='data/Airline_review.csv', 
    include_validation=True, 
    sample_size=5000, 
    stop_words=None, 
    lemmatize=False, 
    max_tokens=10000, 
    percentile_len=0.9, 
    batch_size=64
)
def run_experiment(units, recurrent_type, rnn_layers, dense_layers, runs, train_ds, val_ds, epochs, sequence_length, vocab_size, dropout_rate):
    """
    Runs the experiment for a specific configuration and returns the average metrics.
    """
    model_function = lambda: build_rnn_model(
        rnn_layers=rnn_layers,
        dense_layers=dense_layers,
        recurrent_type=recurrent_type,
        bi_directional=True,
        dropout_rate=dropout_rate,
        units=units,
        sequence_length=sequence_length,
        vocab_size=vocab_size
    )
    return calculate_average_metrics(runs, model_function, train_ds, val_ds, epochs)

# Configuration options
param_dict = {
    'recurrent_type': ['gru', 'lstm'],
    'rnn_layers': [1],
    'dense_layers': [1],
    'units': [256]
}

configurations = generate_configurations(param_dict)

results = []
runs = 2
epochs = 50

# Run experiments
for config in tqdm(configurations,"Running Configurations"):
    metrics = run_experiment(
        units=config['units'],
        recurrent_type=config['recurrent_type'],
        rnn_layers=config['rnn_layers'],
        dense_layers=config['dense_layers'],
        dropout_rate = 0.2,
        runs=runs,
        train_ds=train_ds,
        val_ds=val_ds,
        epochs=epochs,
        sequence_length=sequence_length,
        vocab_size=vocab_size
    )
    results.append({
        **metrics,
        **config  # Unpack configuration into the results
    })

# Create DataFrame and format
df = pd.DataFrame(results)
df = df.round({'loss': 4, 'val_loss': 4, 'val_accuracy': 4, 'val_auc': 4})
df['duration'] = df['duration'].round(0).astype(int)

df

Running Configurations:   0%|          | 0/2 [00:00<?, ?it/s]


Epoch 1/50

63/63 [==============================] - 51s 777ms/step - loss: 0.4822 - accuracy: 0.7607 - auc: 0.8163 - val_loss: 0.3788 - val_accuracy: 0.8240 - val_auc: 0.9054
Epoch 2/50
63/63 [==============================] - 50s 788ms/step - loss: 0.2244 - accuracy: 0.9122 - auc: 0.9643 - val_loss: 0.4816 - val_accuracy: 0.8300 - val_auc: 0.9044
Epoch 3/50
63/63 [==============================] - 49s 772ms/step - loss: 0.1234 - accuracy: 0.9597 - auc: 0.9867 - val_loss: 0.7205 - val_accuracy: 0.8240 - val_auc: 0.8930
Epoch 4/50
63/63 [==============================] - 49s 773ms/step - loss: 0.0772 - accuracy: 0.9747 - auc: 0.9944 - val_loss: 0.7240 - val_accuracy: 0.8080 - val_auc: 0.8716
Epoch 5/50
63/63 [==============================] - 50s 792ms/step - loss: 0.1219 - accuracy: 0.9582 - auc: 0.9901 - val_loss: 0.6939 - val_accuracy: 0.8180 - val_auc: 0.8684
Epoch 6/50
63/63 [==============================] - 49s 783ms/step - loss: 0.0518 - accuracy: 0.9822 - auc: 0.9976 - val_lo

,loss,val_loss,val_accuracy,val_auc,duration,recurrent_type,rnn_layers,dense_layers,units
0,0.4845,0.3743,0.824,0.9066,299,gru,1,1,256
1,0.3477,0.3369,0.864,0.9241,284,lstm,1,1,256


In [6]:
def iterate(sample_size,
            max_tokens,
            percentile_len,
            batch_size,
            param_dict,
            runs,
            epochs):

    train_ds, val_ds, vocab_size, sequence_length, batch_size = load_and_prepare_data(
        path='data/Airline_review.csv', 
        include_validation=True, 
        sample_size=sample_size, 
        stop_words=None, 
        lemmatize=False, 
        max_tokens=max_tokens, 
        percentile_len=percentile_len, 
        batch_size=batch_size
    )
    def run_experiment(units, recurrent_type, rnn_layers, dense_layers, runs, train_ds, val_ds, epochs, sequence_length, vocab_size, dropout_rate, bi_directional):
        """
        Runs the experiment for a specific configuration and returns the average metrics.
        """
        model_function = lambda: build_rnn_model(
            rnn_layers=rnn_layers,
            dense_layers=dense_layers,
            recurrent_type=recurrent_type,
            bi_directional=bi_directional,
            dropout_rate=dropout_rate,
            units=units,
            sequence_length=sequence_length,
            vocab_size=vocab_size
        )
        return calculate_average_metrics(runs, model_function, train_ds, val_ds, epochs)
    
    configurations = generate_configurations(param_dict)
    
    results = []
  
    for config in tqdm(configurations,"Running Configurations"):
        metrics = run_experiment(
            units=config['units'],
            recurrent_type=config['recurrent_type'],
            rnn_layers=config['rnn_layers'],
            dense_layers=config['dense_layers'],
            dropout_rate = config['dropout_rate'],
            bi_directional=config['bi_directional'],
            runs=runs,
            train_ds=train_ds,
            val_ds=val_ds,
            epochs=epochs,
            sequence_length=sequence_length,
            vocab_size=vocab_size
        )
        results.append({
            **metrics,
            **config  # Unpack configuration into the results
        })
    
    # Create DataFrame and format
    df = pd.DataFrame(results)
    df = df.round({'loss': 4, 'val_loss': 4, 'val_accuracy': 4, 'val_auc': 4})
    df['duration'] = df['duration'].round(0).astype(int)
    return df

In [7]:
params = {
    'recurrent_type': ['gru', 'lstm'],
    'rnn_layers': [1],
    'dense_layers': [1],
    'units': [64],
    'dropout_rate': [0.2],
    'bi_directional':[True]}

iterate(sample_size=20000,
        max_tokens=10000,
        percentile_len=0.9,
        batch_size=64,
        param_dict= params,
        runs = 2,
        epochs = 50)

Running Configurations:   0%|          | 0/2 [00:00<?, ?it/s]

Epoch 1/50
250/250 [==============================] - 28s 104ms/step - loss: 0.3703 - accuracy: 0.8313 - auc: 0.9009 - val_loss: 0.2554 - val_accuracy: 0.8940 - val_auc: 0.9546
Epoch 2/50
250/250 [==============================] - 25s 101ms/step - loss: 0.1980 - accuracy: 0.9246 - auc: 0.9717 - val_loss: 0.2723 - val_accuracy: 0.8955 - val_auc: 0.9529
Epoch 3/50
250/250 [==============================] - 25s 99ms/step - loss: 0.1802 - accuracy: 0.9353 - auc: 0.9758 - val_loss: 0.2697 - val_accuracy: 0.9005 - val_auc: 0.9552
Epoch 4/50
250/250 [==============================] - 24s 98ms/step - loss: 0.1390 - accuracy: 0.9506 - auc: 0.9846 - val_loss: 0.4593 - val_accuracy: 0.8910 - val_auc: 0.9221
Epoch 5/50
250/250 [==============================] - 25s 100ms/step - loss: 0.1135 - accuracy: 0.9620 - auc: 0.9890 - val_loss: 0.3579 - val_accuracy: 0.9015 - val_auc: 0.9464
Epoch 6/50
250/250 [==============================] - 25s 100ms/step - loss: 0.1163 - accuracy: 0.9594 - auc: 0.9888 

,loss,val_loss,val_accuracy,val_auc,duration,recurrent_type,rnn_layers,dense_layers,units,dropout_rate,bi_directional
0,0.2964,0.2629,0.8950,0.9536,161,gru,1,1,64,0.2,True
1,0.3552,0.2559,0.8942,0.9600,191,lstm,1,1,64,0.2,True


In [8]:
params = {
    'recurrent_type': ['gru', 'lstm'],
    'rnn_layers': [1],
    'dense_layers': [1],
    'units': [64],
    'dropout_rate': [0.2],
    'bi_directional':[True]}

iterate(sample_size=5000,
        max_tokens=20000,
        percentile_len=0.9,
        batch_size=64,
        param_dict= params,
        runs = 2,
        epochs = 50)

Running Configurations:   0%|          | 0/2 [00:00<?, ?it/s]

Epoch 1/50
63/63 [==============================] - 9s 110ms/step - loss: 0.5846 - accuracy: 0.7023 - auc: 0.6726 - val_loss: 0.3628 - val_accuracy: 0.8440 - val_auc: 0.9219
Epoch 2/50
63/63 [==============================] - 7s 105ms/step - loss: 0.2925 - accuracy: 0.8780 - auc: 0.9411 - val_loss: 0.3066 - val_accuracy: 0.8780 - val_auc: 0.9485
Epoch 3/50
63/63 [==============================] - 7s 104ms/step - loss: 0.1480 - accuracy: 0.9500 - auc: 0.9831 - val_loss: 0.3396 - val_accuracy: 0.8920 - val_auc: 0.9479
Epoch 4/50
63/63 [==============================] - 6s 103ms/step - loss: 0.0690 - accuracy: 0.9818 - auc: 0.9946 - val_loss: 0.4326 - val_accuracy: 0.8820 - val_auc: 0.9361
Epoch 5/50
63/63 [==============================] - 6s 94ms/step - loss: 0.0528 - accuracy: 0.9827 - auc: 0.9975 - val_loss: 0.7267 - val_accuracy: 0.8620 - val_auc: 0.8847
Epoch 6/50
63/63 [==============================] - 6s 103ms/step - loss: 0.0592 - accuracy: 0.9778 - auc: 0.9967 - val_loss: 0.671

,loss,val_loss,val_accuracy,val_auc,duration,recurrent_type,rnn_layers,dense_layers,units,dropout_rate,bi_directional
0,0.3007,0.3027,0.879,0.9465,47,gru,1,1,64,0.2,True
1,0.2024,0.2937,0.880,0.9460,61,lstm,1,1,64,0.2,True


In [10]:
params = {
    'recurrent_type': ['gru', 'lstm'],
    'rnn_layers': [1,2],
    'dense_layers': [1,2],
    'units': [64],
    'dropout_rate': [0.2],
    'bi_directional':[True]}

iterate(sample_size=20000,
        max_tokens=20000,
        percentile_len=0.9,
        batch_size=64,
        param_dict= params,
        runs = 2,
        epochs = 50)

Running Configurations:   0%|          | 0/8 [00:00<?, ?it/s]

Epoch 1/50
250/250 [==============================] - 32s 118ms/step - loss: 0.3684 - accuracy: 0.8345 - auc: 0.9019 - val_loss: 0.2414 - val_accuracy: 0.9075 - val_auc: 0.9589
Epoch 2/50
250/250 [==============================] - 29s 116ms/step - loss: 0.1976 - accuracy: 0.9272 - auc: 0.9711 - val_loss: 0.2835 - val_accuracy: 0.8850 - val_auc: 0.9541
Epoch 3/50
250/250 [==============================] - 29s 117ms/step - loss: 0.1483 - accuracy: 0.9491 - auc: 0.9824 - val_loss: 0.2879 - val_accuracy: 0.8865 - val_auc: 0.9511
Epoch 4/50
250/250 [==============================] - 29s 116ms/step - loss: 0.1187 - accuracy: 0.9617 - auc: 0.9878 - val_loss: 0.2758 - val_accuracy: 0.9005 - val_auc: 0.9568
Epoch 5/50
250/250 [==============================] - 29s 118ms/step - loss: 0.1016 - accuracy: 0.9653 - auc: 0.9911 - val_loss: 0.3125 - val_accuracy: 0.9015 - val_auc: 0.9561
Epoch 6/50
250/250 [==============================] - 29s 117ms/step - loss: 0.1073 - accuracy: 0.9616 - auc: 0.990

,loss,val_loss,val_accuracy,val_auc,duration,recurrent_type,rnn_layers,dense_layers,units,dropout_rate,bi_directional
0,0.3688,0.2425,0.9043,0.9595,178,gru,1,1,64,0.2,True
1,0.3964,0.2462,0.9025,0.9585,185,gru,1,2,64,0.2,True
2,0.3789,0.2507,0.8985,0.9579,316,gru,2,1,64,0.2,True
3,0.3256,0.2497,0.8990,0.9589,324,gru,2,2,64,0.2,True
4,0.3412,0.2316,0.9092,0.9646,228,lstm,1,1,64,0.2,True
5,0.3711,0.2424,0.8997,0.9612,225,lstm,1,2,64,0.2,True
6,0.3697,0.2344,0.9055,0.9636,352,lstm,2,1,64,0.2,True
7,0.2794,0.2488,0.9010,0.9580,422,lstm,2,2,64,0.2,True


In [11]:
params = {
    'recurrent_type': ['gru', 'lstm'],
    'rnn_layers': [1],
    'dense_layers': [1],
    'units': [64],
    'dropout_rate': [0.2],
    'bi_directional':[True]}

iterate(sample_size=5000,
        max_tokens=10000,
        percentile_len=0.99,
        batch_size=64,
        param_dict= params,
        runs = 2,
        epochs = 50)

Running Configurations:   0%|          | 0/2 [00:00<?, ?it/s]

Epoch 1/50
63/63 [==============================] - 18s 252ms/step - loss: 0.5716 - accuracy: 0.7035 - auc: 0.7010 - val_loss: 0.4002 - val_accuracy: 0.8040 - val_auc: 0.8960
Epoch 2/50
63/63 [==============================] - 15s 246ms/step - loss: 0.2741 - accuracy: 0.8907 - auc: 0.9472 - val_loss: 0.3187 - val_accuracy: 0.8600 - val_auc: 0.9294
Epoch 3/50
63/63 [==============================] - 16s 248ms/step - loss: 0.1464 - accuracy: 0.9517 - auc: 0.9830 - val_loss: 0.3732 - val_accuracy: 0.8520 - val_auc: 0.9237
Epoch 4/50
63/63 [==============================] - 15s 246ms/step - loss: 0.0894 - accuracy: 0.9735 - auc: 0.9927 - val_loss: 0.4665 - val_accuracy: 0.8420 - val_auc: 0.9040
Epoch 5/50
63/63 [==============================] - 15s 245ms/step - loss: 0.0482 - accuracy: 0.9845 - auc: 0.9972 - val_loss: 0.5291 - val_accuracy: 0.8380 - val_auc: 0.9015
Epoch 6/50
63/63 [==============================] - 16s 248ms/step - loss: 0.0318 - accuracy: 0.9890 - auc: 0.9992 - val_loss

,loss,val_loss,val_accuracy,val_auc,duration,recurrent_type,rnn_layers,dense_layers,units,dropout_rate,bi_directional
0,0.2913,0.320,0.862,0.9330,113,gru,1,1,64,0.2,True
1,0.2637,0.303,0.877,0.9464,130,lstm,1,1,64,0.2,True


In [12]:
params = {
    'recurrent_type': ['gru', 'lstm'],
    'rnn_layers': [1],
    'dense_layers': [1],
    'units': [64],
    'dropout_rate': [0.2],
    'bi_directional':[True]}

iterate(sample_size=5000,
        max_tokens=10000,
        percentile_len=0.85,
        batch_size=64,
        param_dict= params,
        runs = 2,
        epochs = 50)

Running Configurations:   0%|          | 0/2 [00:00<?, ?it/s]

Epoch 1/50
63/63 [==============================] - 8s 88ms/step - loss: 0.5635 - accuracy: 0.7100 - auc: 0.7335 - val_loss: 0.3811 - val_accuracy: 0.8420 - val_auc: 0.9090
Epoch 2/50
63/63 [==============================] - 5s 78ms/step - loss: 0.2670 - accuracy: 0.8978 - auc: 0.9511 - val_loss: 0.3338 - val_accuracy: 0.8600 - val_auc: 0.9325
Epoch 3/50
63/63 [==============================] - 5s 80ms/step - loss: 0.1366 - accuracy: 0.9542 - auc: 0.9858 - val_loss: 0.3976 - val_accuracy: 0.8640 - val_auc: 0.9242
Epoch 4/50
63/63 [==============================] - 5s 79ms/step - loss: 0.0843 - accuracy: 0.9730 - auc: 0.9940 - val_loss: 0.4851 - val_accuracy: 0.8600 - val_auc: 0.9231
Epoch 5/50
63/63 [==============================] - 5s 79ms/step - loss: 0.0519 - accuracy: 0.9843 - auc: 0.9975 - val_loss: 0.6315 - val_accuracy: 0.8380 - val_auc: 0.9158
Epoch 6/50
63/63 [==============================] - 5s 78ms/step - loss: 0.0415 - accuracy: 0.9870 - auc: 0.9985 - val_loss: 0.6759 - v

,loss,val_loss,val_accuracy,val_auc,duration,recurrent_type,rnn_layers,dense_layers,units,dropout_rate,bi_directional
0,0.2678,0.3400,0.872,0.9344,38,gru,1,1,64,0.2,True
1,0.2631,0.3041,0.876,0.9390,50,lstm,1,1,64,0.2,True


In [13]:
params = {
    'recurrent_type': ['gru'],
    'rnn_layers': [1,2],
    'dense_layers': [1,2],
    'units': [32,16],
    'dropout_rate': [0],
    'bi_directional':[True]}

iterate(sample_size=5000,
        max_tokens=10000,
        percentile_len=0.8,
        batch_size=64,
        param_dict= params,
        runs = 2,
        epochs = 50)

Running Configurations:   0%|          | 0/8 [00:00<?, ?it/s]

Epoch 1/50
63/63 [==============================] - 5s 47ms/step - loss: 0.6304 - accuracy: 0.6607 - auc: 0.5800 - val_loss: 0.4803 - val_accuracy: 0.7060 - val_auc: 0.9002
Epoch 2/50
63/63 [==============================] - 3s 41ms/step - loss: 0.3383 - accuracy: 0.8580 - auc: 0.9222 - val_loss: 0.3162 - val_accuracy: 0.8620 - val_auc: 0.9413
Epoch 3/50
63/63 [==============================] - 3s 40ms/step - loss: 0.1867 - accuracy: 0.9252 - auc: 0.9760 - val_loss: 0.2739 - val_accuracy: 0.8840 - val_auc: 0.9475
Epoch 4/50
63/63 [==============================] - 3s 40ms/step - loss: 0.1173 - accuracy: 0.9613 - auc: 0.9887 - val_loss: 0.3731 - val_accuracy: 0.8320 - val_auc: 0.9261
Epoch 5/50
63/63 [==============================] - 2s 39ms/step - loss: 0.0883 - accuracy: 0.9712 - auc: 0.9935 - val_loss: 0.3766 - val_accuracy: 0.8840 - val_auc: 0.9362
Epoch 6/50
63/63 [==============================] - 3s 42ms/step - loss: 0.0594 - accuracy: 0.9805 - auc: 0.9963 - val_loss: 0.4776 - v

,loss,val_loss,val_accuracy,val_auc,duration,recurrent_type,rnn_layers,dense_layers,units,dropout_rate,bi_directional
0,0.2723,0.2860,0.885,0.9422,22,gru,1,1,32,0,True
1,0.2600,0.2764,0.890,0.9498,21,gru,1,1,16,0,True
2,0.2240,0.2945,0.880,0.9416,24,gru,1,2,32,0,True
3,0.2893,0.2992,0.892,0.9431,21,gru,1,2,16,0,True
4,0.3344,0.2877,0.884,0.9449,41,gru,2,1,32,0,True
5,0.3121,0.3003,0.896,0.9406,41,gru,2,1,16,0,True
6,0.2542,0.3426,0.874,0.9303,46,gru,2,2,32,0,True
7,0.2549,0.2979,0.883,0.9378,44,gru,2,2,16,0,True


**changing min_delta to 0.001**

In [15]:
params = {
    'recurrent_type': ['gru'],
    'rnn_layers': [1,2],
    'dense_layers': [1,2],
    'units': [32,16],
    'dropout_rate': [0],
    'bi_directional':[True]}

iterate(sample_size=5000,
        max_tokens=10000,
        percentile_len=0.8,
        batch_size=64,
        param_dict= params,
        runs = 2,
        epochs = 50)

Running Configurations:   0%|          | 0/8 [00:00<?, ?it/s]

,loss,val_loss,val_accuracy,val_auc,duration,recurrent_type,rnn_layers,dense_layers,units,dropout_rate,bi_directional
0,0.3251,0.3021,0.879,0.9426,21,gru,1,1,32,0,True
1,0.2024,0.2961,0.884,0.9437,24,gru,1,1,16,0,True
2,0.1790,0.3119,0.873,0.9428,25,gru,1,2,32,0,True
3,0.2970,0.3694,0.867,0.9229,24,gru,1,2,16,0,True
4,0.3498,0.3210,0.864,0.9288,41,gru,2,1,32,0,True
5,0.2298,0.2992,0.886,0.9440,46,gru,2,1,16,0,True
6,0.2364,0.3185,0.872,0.9356,49,gru,2,2,32,0,True
7,0.4393,0.4674,0.772,0.7193,76,gru,2,2,16,0,True


In [16]:
params = {
    'recurrent_type': ['gru'],
    'rnn_layers': [1,2],
    'dense_layers': [1,2],
    'units': [32,16],
    'dropout_rate': [0],
    'bi_directional':[True]}

iterate(sample_size=5000,
        max_tokens=20000,
        percentile_len=0.8,
        batch_size=64,
        param_dict= params,
        runs = 3,
        epochs = 50)

Running Configurations:   0%|          | 0/8 [00:00<?, ?it/s]

,loss,val_loss,val_accuracy,val_auc,duration,recurrent_type,rnn_layers,dense_layers,units,dropout_rate,bi_directional
0,0.2892,0.3010,0.8827,0.9445,22,gru,1,1,32,0,True
1,0.2623,0.2875,0.8900,0.9511,22,gru,1,1,16,0,True
2,0.3274,0.3231,0.8733,0.9433,22,gru,1,2,32,0,True
3,0.3335,0.4162,0.8693,0.9035,29,gru,1,2,16,0,True
4,0.2288,0.3046,0.8813,0.9466,45,gru,2,1,32,0,True
5,0.2145,0.3245,0.8813,0.9452,46,gru,2,1,16,0,True
6,0.3175,0.3234,0.8787,0.9409,44,gru,2,2,32,0,True
7,0.5455,0.5602,0.7273,0.6393,75,gru,2,2,16,0,True


In [18]:
params = {
    'recurrent_type': ['gru'],
    'rnn_layers': [1,2],
    'dense_layers': [1,2],
    'units': [32,16,8],
    'dropout_rate': [0],
    'bi_directional':[True]}

iterate(sample_size=10000,
        max_tokens=20000,
        percentile_len=0.8,
        batch_size=64,
        param_dict= params,
        runs = 3,
        epochs = 50)

Running Configurations:   0%|          | 0/12 [00:00<?, ?it/s]

,loss,val_loss,val_accuracy,val_auc,duration,recurrent_type,rnn_layers,dense_layers,units,dropout_rate,bi_directional
0,0.2063,0.2494,0.8997,0.9576,43,gru,1,1,32,0,True
1,0.2688,0.2457,0.8977,0.9578,36,gru,1,1,16,0,True
2,0.2136,0.2561,0.8957,0.9532,36,gru,1,1,8,0,True
3,0.2409,0.2595,0.8980,0.9568,42,gru,1,2,32,0,True
4,0.2604,0.2927,0.8917,0.9537,41,gru,1,2,16,0,True
5,0.1696,0.2975,0.8893,0.9425,50,gru,1,2,8,0,True
6,0.2029,0.2492,0.9040,0.9594,83,gru,2,1,32,0,True
7,0.2914,0.2580,0.8963,0.9534,69,gru,2,1,16,0,True
8,0.2938,0.4316,0.8113,0.7892,119,gru,2,1,8,0,True
9,0.2497,0.2733,0.8980,0.9573,84,gru,2,2,32,0,True


In [19]:
params = {
    'recurrent_type': ['gru'],
    'rnn_layers': [1],
    'dense_layers': [1],
    'units': [32,16,8],
    'dropout_rate': [0, 0.1, 0.2, 0.3, 0.4, 0.5],
    'bi_directional':[True]}

iterate(sample_size=10000,
        max_tokens=20000,
        percentile_len=0.8,
        batch_size=64,
        param_dict= params,
        runs = 3,
        epochs = 50)

Running Configurations:   0%|          | 0/18 [00:00<?, ?it/s]

,loss,val_loss,val_accuracy,val_auc,duration,recurrent_type,rnn_layers,dense_layers,units,dropout_rate,bi_directional
0,0.2056,0.2875,0.8797,0.9513,41,gru,1,1,32,0.0,True
1,0.2353,0.2692,0.8870,0.9535,40,gru,1,1,32,0.1,True
2,0.3410,0.2889,0.8803,0.9492,38,gru,1,1,32,0.2,True
3,0.2525,0.2727,0.8880,0.9538,40,gru,1,1,32,0.3,True
4,0.2657,0.2897,0.8887,0.9511,40,gru,1,1,32,0.4,True
5,0.2636,0.2839,0.8883,0.9526,41,gru,1,1,32,0.5,True
6,0.2342,0.2724,0.8853,0.9530,37,gru,1,1,16,0.0,True
7,0.2703,0.2835,0.8777,0.9494,37,gru,1,1,16,0.1,True
8,0.2427,0.2769,0.8870,0.9530,40,gru,1,1,16,0.2,True
9,0.3146,0.2738,0.8843,0.9527,37,gru,1,1,16,0.3,True
